# RP3beta

## Import

In [1]:
## Allow more than one output for a single code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
import scipy.sparse as sps
import numpy as np
import os

## Set the numpy random seed
SEED = 42
np.random.seed(SEED)

os.getcwd()

'/home/alessio/Scrivania/RecSysChallenge2021'

In [3]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

from Evaluation.Evaluator import EvaluatorHoldout

from Recommenders.Recommender_import_list import *

from Recommenders.DataIO import DataIO

Tensorflow is not available


In [4]:
## Utility Functions
from Dataset.load_data import load_data

## Data Loading and Split

In [5]:
URM_all, ICM_dict = load_data()

In [6]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.90)

URM_aug_train = sps.vstack([URM_train.copy().tocoo(), 
                            ICM_dict['ICM_genre'].T.tocoo(),
                            ICM_dict['ICM_subgenre'].T.tocoo(), 
                            #ICM_dict['ICM_event'].T.tocoo(), 
                            ICM_dict['ICM_channel'].T.tocoo()], format='csr')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], exclude_seen = True)

EvaluatorHoldout: Ignoring 13610 ( 0.3%) Users that have less than 1 test interactions


## Optimization

In [9]:
!ls "./result_experiments/P3alpha"

P3alpha_augmented_subgenre_channel
P3alpha_augmented_subgenre_channel2
P3alpha_augmented_subgenre_channel_improved_range
P3alpha_augmented_subgenre_channel_range_improved
P3alpha_augmented_subgenre_channel_range_improved_topK_categorical
P3alpha_augmented_subgenre_channel_range_improved_topK_categorical_improved


In [6]:
output_folder_path = "result_experiments/P3alpha/P3alpha_augmented_subgenre_channel/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(P3alphaRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.20

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,normalize_similarity,MAP
28,126,0.765545,False,0.206943
38,202,0.731517,False,0.201909
61,85,0.711881,False,0.206780
96,115,0.740923,False,0.206167
97,281,0.809203,False,0.202563


Max value for the range:  281
Min value for the range:  85
Best MAP:  0.20694283050559711


,topK,alpha,normalize_similarity,MAP
28,126,0.765545,False,0.206943


In [10]:
output_folder_path = "result_experiments/P3alpha/P3alpha_augmented_subgenre_channel2/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(P3alphaRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.20

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,normalize_similarity,MAP
13,155,0.858439,False,0.204662
31,200,0.796338,False,0.205217
32,103,0.771157,False,0.208743
34,50,0.598614,False,0.201410
37,50,0.819312,False,0.206084
39,50,0.72698,False,0.207122
41,120,0.810202,False,0.207707
43,50,0.772735,False,0.208846
48,197,0.912647,False,0.204422
50,200,0.864395,False,0.203091


Max value for the range:  200
Min value for the range:  50
Best MAP:  0.2100361958501699


,topK,alpha,normalize_similarity,MAP
97,79,0.777344,False,0.210036


In [8]:
output_folder_path = "result_experiments/P3alpha/P3alpha_augmented_subgenre_channel_improved_range/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(P3alphaRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.21

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,normalize_similarity,MAP
67,84,0.764591,False,0.210027
69,82,0.766348,False,0.210205
72,82,0.7627,False,0.210047
73,81,0.759588,False,0.210037


Max value for the range:  84
Min value for the range:  81
Best MAP:  0.21020464162996824


,topK,alpha,normalize_similarity,MAP
69,82,0.766348,False,0.210205


In [12]:
output_folder_path = "result_experiments/P3alpha/P3alpha_augmented_subgenre_channel_range_improved/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(P3alphaRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.20

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,normalize_similarity,MAP
3,73,0.600505,False,0.202164
23,87,0.663257,False,0.204420
27,89,0.868611,False,0.207466
30,70,0.769899,False,0.208976
36,70,0.836186,False,0.206567
38,70,0.717404,False,0.207818
40,70,0.76239,False,0.209213
41,70,0.762123,False,0.209207
42,70,0.762368,False,0.209213
43,95,0.747034,False,0.208073


Max value for the range:  95
Min value for the range:  70
Best MAP:  0.2098957831703699


,topK,alpha,normalize_similarity,MAP
83,86,0.786711,False,0.209896


In [14]:
output_folder_path = "result_experiments/P3alpha/P3alpha_augmented_subgenre_channel_range_improved_topK_categorical/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(P3alphaRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.205

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,normalize_similarity,MAP
22,85,0.82565,False,0.206599
30,92,0.717894,False,0.206907
33,77,0.771217,False,0.209453
36,92,0.792513,False,0.209345
37,92,0.753996,False,0.208368
44,77,0.794549,False,0.209038
45,92,0.778725,False,0.209150
46,77,0.781964,False,0.209443
47,77,0.781476,False,0.209424
48,77,0.780633,False,0.209532


Max value for the range:  92
Min value for the range:  77
Best MAP:  0.20978944641025096


,topK,alpha,normalize_similarity,MAP
87,77,0.778855,False,0.209789


In [16]:
output_folder_path = "result_experiments/P3alpha/P3alpha_augmented_subgenre_channel_range_improved_topK_categorical_improved/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(P3alphaRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.21

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,normalize_similarity,MAP
4,79,0.762557,False,0.210177
62,79,0.765651,False,0.210324
63,79,0.766522,False,0.210370
64,79,0.767193,False,0.210340
65,79,0.767415,False,0.210310
67,79,0.766726,False,0.210304
68,79,0.765943,False,0.210386
70,79,0.767875,False,0.210122
72,79,0.762245,False,0.210178
74,79,0.761078,False,0.210094


Max value for the range:  79
Min value for the range:  79
Best MAP:  0.2103869120319002


,topK,alpha,normalize_similarity,MAP
89,79,0.767175,False,0.210387


In [23]:
output_folder_path = "result_experiments/prova/" # topK from 50 to 200

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 100  # 50 with 30% random is a good number
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [ ]:
from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative

runHyperparameterSearch_Collaborative(P3alphaRecommender,
                                      URM_train = URM_aug_train,
                                      URM_train_last_test = None,
                                      metric_to_optimize = metric_to_optimize,
                                      cutoff_to_optimize = cutoff_to_optimize,
                                      n_cases = n_cases,
                                      n_random_starts = n_random_starts,
                                      evaluator_validation_earlystopping = evaluator_validation,
                                      evaluator_validation = evaluator_validation,
                                      evaluator_test = None,
                                      output_folder_path = output_folder_path,
                                      resume_from_saved = True,
                                      similarity_type_list = None,
                                      parallelizeKNN = True)